In [ ]:
from astroquery.mast import Catalogs

star = Catalogs.query_object(f"TIC {tic}", catalog="TIC", radius=0.001)
tic_rho_star = float(star["rho"]), float(star["e_rho"])
if np.all(np.isfinite(tic_rho_star)):

    print("rho_star = {0} ± {1}".format(*tic_rho_star))

    # Extract the implied density from the fit
    rho_circ = np.repeat(trace["rho_circ"], 500, axis=0)
    period = np.repeat(trace["period"], 500, axis=0)

    # Sample eccentricity and omega uniformly
    ecc = np.random.uniform(0, 1, len(rho_circ))
    omega = np.random.uniform(-np.pi, np.pi, len(rho_circ))

    # Compute the "g" parameter from Dawson & Johnson and what true
    # density that implies
    g = (1 + ecc * np.sin(omega)) / np.sqrt(1 - ecc ** 2)
    rho = rho_circ / g[:, None] ** 3

    # Re-weight these samples to get weighted posterior samples
    log_weights = -0.5 * ((rho - tic_rho_star[0]) / tic_rho_star[1]) ** 2

    for n in range(num_toi):
        weights = np.exp(log_weights[:, n] - np.max(log_weights[:, n]))
        if single_transit[n]:
            samples = np.vstack((ecc, omega, np.log10(period[:, n]))).T
            labels = [
                f"$e_{n + 1}$",
                f"$\omega_{n + 1}$",
                f"$\log_{{10}} P_{n + 1} / \mathrm{{day}}$",
            ]
        else:
            samples = np.vstack((ecc, omega)).T
            labels = [f"$e_{n + 1}$", f"$\omega_{n + 1}$"]

        fig = corner.corner(
            samples, weights=weights, plot_datapoints=False, labels=labels,
        )
        fig.suptitle(f"TOI {toi_num}.{n + 1:02d}")

else:
    print("The TIC has no measured density")